# Gensim Training Experiments
- **Where:** HEIA-FR GPU-2
- **Dataset:** enwiki-latest-pages-articles.xml.bz2 (16GB)
- **Dictionary:** enwiki-english-wordids.txt.bz2 (16MB)

## What's going on
- Training a Word2Vec on the full wikipedia english dataset using its full pre-extracted dictionary.

## Results
- **Timeframe:**
    - 32 cores: ~17h
- **Errors:** kernel crash when *resetting layer weights*
- **Comments:**
    - Those 17 hours of training are lost as the model wasn't save before the crash

In [1]:
# Set variables
chunk_pages = 99999
chunk_part = 1
lemmatization = True
datafile_name = "datasets/chunks/enwiki-chunks-"+str(chunk_pages)+"/enwiki-chunk-"+str(chunk_pages)+"-"+str(chunk_part)+".xml.bz2"
dictionary_name = 'dictionaries/enwiki-english-lemmatized.txt.bz2'
model_name = "models/enwiki-full-dict-part-"+str(chunk_pages)+".model"

#datafile_name = "gensim-data/text8/text8.gz"
#dictionary_name = 'dictionaries/my_dict.dict'
#model_name = 'models/mymodel.model'

w2v_size = 300
w2v_window = 5
w2v_min_count = 1
#w2v_cores = multiprocessing.cpu_count()-10
w2v_cores = 10

In [2]:
import os
SCRIPT_PATH = os.path.dirname(os.path.realpath("__file__"))
DATA_PATH   = os.path.join(SCRIPT_PATH, 'datasets/')
DICTIONARY_PATH = os.path.join(SCRIPT_PATH, 'dictionaries/')
MODEL_PATH  = os.path.join(SCRIPT_PATH, 'models/')

DICTIONARY_FILEPATH = os.path.join(DICTIONARY_PATH, dictionary_name)
WIKI_DUMP_FILEPATH = os.path.join(DATA_PATH, datafile_name)

In [3]:
# Start logging process at root level
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logging.root.setLevel(level=logging.INFO)

In [4]:
# Load dictionary from file
from gensim.corpora import Dictionary
dictionary = Dictionary.load_from_text(dictionary_name)

2019-04-02 13:09:56,747 : INFO : 'pattern' package found; tag filters are available for English


In [5]:
# Build WikiCorpus based on the dictionary
from gensim.corpora import WikiCorpus
wiki = WikiCorpus(datafile_name, dictionary=dictionary,lemmatize=lemmatization)

In [6]:
# Initialize simple sentence iterator required for the Word2Vec model
# Bypass memory errors
class SentencesIterator:
    def __init__(self, wiki):
        self.wiki = wiki

    def __iter__(self):
        for sentence in self.wiki.get_texts():
            yield list(map(lambda x: x.decode('utf-8'), sentence)) #set encode('utf-8') when lemmatized = False
            #yield gensim.utils.simple_preprocess(line)

sentences = SentencesIterator(wiki)

In [7]:
# Train model
from gensim.models import Word2Vec
import multiprocessing

print("Running with: " + str(w2v_cores) + " cores")
model = Word2Vec(sentences=sentences,
                 size=w2v_size,
                 min_count=w2v_min_count,
                 window=w2v_window,
                 workers=w2v_cores
                )
model.save(model_name)

del model
del wiki
del sentences
del dictionary

2019-04-02 13:10:09,441 : INFO : collecting all words and their counts


Running with: 10 cores


2019-04-02 13:10:14,925 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-04-02 13:16:13,256 : INFO : PROGRESS: at sentence #10000, processed 21276586 words, keeping 552988 word types
2019-04-02 13:21:35,804 : INFO : PROGRESS: at sentence #20000, processed 39991911 words, keeping 814215 word types
2019-04-02 13:26:24,778 : INFO : PROGRESS: at sentence #30000, processed 55508518 words, keeping 1005642 word types
2019-04-02 13:30:41,055 : INFO : PROGRESS: at sentence #40000, processed 68725549 words, keeping 1174574 word types
2019-04-02 13:33:38,517 : INFO : PROGRESS: at sentence #50000, processed 78186601 words, keeping 1276212 word types
2019-04-02 13:35:06,101 : INFO : PROGRESS: at sentence #60000, processed 83639646 words, keeping 1303378 word types
2019-04-02 13:36:25,357 : INFO : PROGRESS: at sentence #70000, processed 88508149 words, keeping 1328620 word types
2019-04-02 13:37:14,977 : INFO : finished iterating over Wikipedia corpus of 76767 document

2019-04-02 13:39:33,473 : INFO : EPOCH 1 - PROGRESS: at 3.55% examples, 50231 words/s, in_qsize 0, out_qsize 0
2019-04-02 13:39:35,824 : INFO : EPOCH 1 - PROGRESS: at 3.56% examples, 49201 words/s, in_qsize 0, out_qsize 0
2019-04-02 13:39:37,630 : INFO : EPOCH 1 - PROGRESS: at 3.67% examples, 49887 words/s, in_qsize 0, out_qsize 0
2019-04-02 13:39:38,660 : INFO : EPOCH 1 - PROGRESS: at 3.71% examples, 49782 words/s, in_qsize 0, out_qsize 0
2019-04-02 13:39:41,462 : INFO : EPOCH 1 - PROGRESS: at 3.85% examples, 50221 words/s, in_qsize 0, out_qsize 0
2019-04-02 13:39:42,722 : INFO : EPOCH 1 - PROGRESS: at 3.88% examples, 50149 words/s, in_qsize 2, out_qsize 1
2019-04-02 13:39:45,765 : INFO : EPOCH 1 - PROGRESS: at 3.95% examples, 50006 words/s, in_qsize 0, out_qsize 0
2019-04-02 13:39:47,502 : INFO : EPOCH 1 - PROGRESS: at 3.97% examples, 49473 words/s, in_qsize 0, out_qsize 0
2019-04-02 13:39:50,756 : INFO : EPOCH 1 - PROGRESS: at 4.12% examples, 50149 words/s, in_qsize 0, out_qsize 0
2

2019-04-02 13:41:52,262 : INFO : EPOCH 1 - PROGRESS: at 8.36% examples, 51230 words/s, in_qsize 0, out_qsize 0
2019-04-02 13:41:53,629 : INFO : EPOCH 1 - PROGRESS: at 8.41% examples, 51337 words/s, in_qsize 0, out_qsize 0
2019-04-02 13:41:57,762 : INFO : EPOCH 1 - PROGRESS: at 8.42% examples, 50562 words/s, in_qsize 0, out_qsize 0
2019-04-02 13:42:02,191 : INFO : EPOCH 1 - PROGRESS: at 8.47% examples, 50161 words/s, in_qsize 1, out_qsize 0
2019-04-02 13:42:07,731 : INFO : EPOCH 1 - PROGRESS: at 8.66% examples, 50395 words/s, in_qsize 0, out_qsize 0
2019-04-02 13:42:09,071 : INFO : EPOCH 1 - PROGRESS: at 8.83% examples, 51211 words/s, in_qsize 0, out_qsize 0
2019-04-02 13:42:10,142 : INFO : EPOCH 1 - PROGRESS: at 8.84% examples, 51075 words/s, in_qsize 1, out_qsize 0
2019-04-02 13:42:11,704 : INFO : EPOCH 1 - PROGRESS: at 8.89% examples, 51073 words/s, in_qsize 0, out_qsize 0
2019-04-02 13:42:21,213 : INFO : EPOCH 1 - PROGRESS: at 8.94% examples, 49629 words/s, in_qsize 2, out_qsize 0
2

...

2019-04-02 15:50:14,914 : INFO : EPOCH 5 - PROGRESS: at 50.50% examples, 51234 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:50:17,227 : INFO : EPOCH 5 - PROGRESS: at 50.52% examples, 51150 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:50:18,712 : INFO : EPOCH 5 - PROGRESS: at 50.78% examples, 51248 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:50:20,378 : INFO : EPOCH 5 - PROGRESS: at 50.88% examples, 51221 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:50:21,727 : INFO : EPOCH 5 - PROGRESS: at 51.00% examples, 51233 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:50:23,148 : INFO : EPOCH 5 - PROGRESS: at 51.08% examples, 51210 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:50:26,080 : INFO : EPOCH 5 - PROGRESS: at 51.10% examples, 51107 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:50:27,665 : INFO : EPOCH 5 - PROGRESS: at 51.24% examples, 51121 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:50:29,266 : INFO : EPOCH 5 - PROGRESS: at 51.48% examples, 51139 words/s, in_qsize 0, out_

2019-04-02 15:52:10,145 : INFO : EPOCH 5 - PROGRESS: at 58.14% examples, 50870 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:52:11,337 : INFO : EPOCH 5 - PROGRESS: at 58.43% examples, 50947 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:52:12,759 : INFO : EPOCH 5 - PROGRESS: at 58.44% examples, 50900 words/s, in_qsize 0, out_qsize 1
2019-04-02 15:52:14,862 : INFO : EPOCH 5 - PROGRESS: at 58.66% examples, 50922 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:52:17,047 : INFO : EPOCH 5 - PROGRESS: at 58.69% examples, 50852 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:52:18,084 : INFO : EPOCH 5 - PROGRESS: at 58.73% examples, 50837 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:52:19,142 : INFO : EPOCH 5 - PROGRESS: at 58.81% examples, 50828 words/s, in_qsize 3, out_qsize 1
2019-04-02 15:52:21,227 : INFO : EPOCH 5 - PROGRESS: at 58.97% examples, 50851 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:52:24,373 : INFO : EPOCH 5 - PROGRESS: at 59.16% examples, 50844 words/s, in_qsize 1, out_

2019-04-02 15:54:02,079 : INFO : EPOCH 5 - PROGRESS: at 67.56% examples, 50911 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:54:03,134 : INFO : EPOCH 5 - PROGRESS: at 67.58% examples, 50879 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:54:04,259 : INFO : EPOCH 5 - PROGRESS: at 67.97% examples, 50935 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:54:05,674 : INFO : EPOCH 5 - PROGRESS: at 68.16% examples, 50931 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:54:07,201 : INFO : EPOCH 5 - PROGRESS: at 68.35% examples, 50932 words/s, in_qsize 0, out_qsize 1
2019-04-02 15:54:08,373 : INFO : EPOCH 5 - PROGRESS: at 68.80% examples, 50973 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:54:09,492 : INFO : EPOCH 5 - PROGRESS: at 69.03% examples, 50983 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:54:10,715 : INFO : EPOCH 5 - PROGRESS: at 69.15% examples, 50965 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:54:12,135 : INFO : EPOCH 5 - PROGRESS: at 69.25% examples, 50938 words/s, in_qsize 0, out_

2019-04-02 15:55:47,572 : INFO : EPOCH 5 - PROGRESS: at 86.01% examples, 51166 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:55:48,719 : INFO : EPOCH 5 - PROGRESS: at 86.04% examples, 51139 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:55:49,906 : INFO : EPOCH 5 - PROGRESS: at 86.11% examples, 51144 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:55:51,382 : INFO : EPOCH 5 - PROGRESS: at 86.13% examples, 51123 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:55:52,969 : INFO : EPOCH 5 - PROGRESS: at 86.17% examples, 51102 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:55:54,516 : INFO : EPOCH 5 - PROGRESS: at 86.18% examples, 51062 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:55:55,542 : INFO : EPOCH 5 - PROGRESS: at 86.40% examples, 51149 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:55:56,655 : INFO : EPOCH 5 - PROGRESS: at 86.50% examples, 51162 words/s, in_qsize 0, out_qsize 0
2019-04-02 15:55:58,743 : INFO : EPOCH 5 - PROGRESS: at 86.55% examples, 51122 words/s, in_qsize 1, out_

2019-04-02 15:57:20,392 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-04-02 15:57:20,393 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-04-02 15:57:20,394 : INFO : EPOCH - 5 : training on 91436297 raw words (85733682 effective words) took 1670.7s, 51317 effective words/s
2019-04-02 15:57:20,396 : INFO : training on a 457181485 raw words (428669249 effective words) took 8374.7s, 51186 effective words/s
2019-04-02 15:57:20,398 : INFO : saving Word2Vec object under models/enwiki-full-dict-part-99999.model, separately None
2019-04-02 15:57:20,400 : INFO : storing np array 'syn1neg' to models/enwiki-full-dict-part-99999.model.trainables.syn1neg.npy
2019-04-02 15:57:21,570 : INFO : not storing attribute vectors_norm
2019-04-02 15:57:21,572 : INFO : storing np array 'vectors' to models/enwiki-full-dict-part-99999.model.wv.vectors.npy
2019-04-02 15:57:22,877 : INFO : not storing attribute cum_table
2019-04-02 15:57:27,011 : INFO : saved models